In [1]:
import pandas as pd

In [2]:
takehome_users = pd.read_csv('takehome_users.csv', 
                             encoding='latin-1', 
                             parse_dates=['creation_time'],
                             infer_datetime_format=True)

In [3]:
takehome_users.head(5)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
takehome_users['last_session_creation_timestamp'] = pd.to_datetime(takehome_users['last_session_creation_time'], unit='s')

In [5]:
takehome_users['invited_by_user'] = takehome_users['invited_by_user_id'].notna() * 1

In [7]:
takehome_users['last_session_happened'] = takehome_users['last_session_creation_timestamp'].notna() * 1

In [8]:
takehome_users.head(5)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,last_session_creation_timestamp,invited_by_user,last_session,last_session_happened
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,2014-04-22 03:53:30,1,1,1
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2014-03-31 03:45:04,1,1,1
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,2013-03-19 23:14:52,1,1,1
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,2013-05-22 08:09:28,1,1,1
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,2013-01-22 10:14:20,1,1,1


In [9]:
takehome_user_engagement = pd.read_csv('takehome_user_engagement.csv',
                                      parse_dates=['time_stamp'])

In [10]:
takehome_user_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [11]:
rolling_7_day_engagement = takehome_user_engagement.groupby(by='user_id').rolling('7d', on='time_stamp').count()

In [12]:
adopted_user_data = rolling_7_day_engagement[rolling_7_day_engagement['visited'] >= 3]

In [13]:
adopted_ids = set(adopted_user_data.index.get_level_values(0))

In [14]:
takehome_users['adopted'] = takehome_users.apply(lambda x: int(x['object_id'] in adopted_ids), axis=1)

In [15]:
takehome_users = pd.get_dummies(takehome_users, columns=['creation_source'])

In [16]:
takehome_users.head(5)

,object_id,creation_time,name,email,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,last_session_creation_timestamp,invited_by_user,last_session,last_session_happened,adopted,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,1.398139e+09,1,0,11,10803.0,2014-04-22 03:53:30,1,1,1,0,1,0,0,0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,1.396238e+09,0,0,1,316.0,2014-03-31 03:45:04,1,1,1,1,0,1,0,0,0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,1.363735e+09,0,0,94,1525.0,2013-03-19 23:14:52,1,1,1,0,0,1,0,0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,1.369210e+09,0,0,1,5151.0,2013-05-22 08:09:28,1,1,1,0,1,0,0,0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,1.358850e+09,0,0,193,5240.0,2013-01-22 10:14:20,1,1,1,0,1,0,0,0,0


In [17]:
y = takehome_users['adopted']
y

0        0
1        1
2        0
3        0
4        0
        ..
11995    0
11996    0
11997    0
11998    0
11999    0
Name: adopted, Length: 12000, dtype: int64

In [18]:
y.value_counts()

0    10398
1     1602
Name: adopted, dtype: int64

In [19]:
features = ['opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id', 'invited_by_user', 
            'creation_source_GUEST_INVITE', 'creation_source_ORG_INVITE',
            'creation_source_PERSONAL_PROJECTS', 'creation_source_SIGNUP', 
            'creation_source_SIGNUP_GOOGLE_AUTH']
X = takehome_users[features]
X.head()

,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1,0,11,1,1,0,0,0,0
1,0,0,1,1,0,1,0,0,0
2,0,0,94,1,0,1,0,0,0
3,0,0,1,1,1,0,0,0,0
4,0,0,193,1,1,0,0,0,0


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=23)

In [22]:
param_grid = {'n_estimators': [50, 100, 200],
              'max_depth': [2, 5, None],
              'min_samples_leaf': [2, 3, 5],
              'criterion': ['gini', 'entropy']
             }

In [23]:
classifier = RandomForestClassifier()

In [24]:
cv = GridSearchCV(estimator=classifier, param_grid=param_grid, n_jobs=-1, verbose=5)

In [25]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:   13.5s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [26]:
cv.cv_results_

{'mean_fit_time': array([0.18408952, 0.36381049, 0.69347334, 0.18322778, 0.34373851,
        0.66551099, 0.17214432, 0.32867017, 0.67655392, 0.21971827,
        0.44288673, 0.82903581, 0.20749331, 0.40331511, 0.83224559,
        0.20276957, 0.40663705, 0.81079855, 0.37980213, 0.75542283,
        1.49889822, 0.3657238 , 0.7248436 , 1.44874082, 0.3422266 ,
        0.68270388, 1.40755763, 0.16137419, 0.33458233, 0.66947131,
        0.16812043, 0.33252797, 0.67006297, 0.17059479, 0.33897567,
        0.6715776 , 0.22783041, 0.43826437, 0.86812057, 0.22167468,
        0.43266749, 0.87757492, 0.22112694, 0.44749584, 0.90930061,
        0.49851575, 1.02176046, 2.13027306, 0.49708891, 1.025737  ,
        2.06518946, 0.48958058, 0.95463719, 1.38312287]),
 'std_fit_time': array([0.00287976, 0.00235552, 0.00541789, 0.00359985, 0.0089509 ,
        0.00657053, 0.00724009, 0.00382526, 0.00921906, 0.00672073,
        0.00369568, 0.01005816, 0.00640023, 0.00333866, 0.01755359,
        0.00182379, 0.002

In [27]:
best_estimator = cv.best_estimator_
best_estimator

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
best_estimator.score(X_train, y_train)

0.8664583333333333

In [35]:
best_estimator.score(X_test, y_test)

0.8666666666666667

In [31]:
list(zip(features, best_estimator.feature_importances_))

[('opted_in_to_mailing_list', 0.004804437731065638),
 ('enabled_for_marketing_drip', 0.011062009796962724),
 ('org_id', 0.4541378909941495),
 ('invited_by_user', 0.06769041645179677),
 ('creation_source_GUEST_INVITE', 0.09653280800928216),
 ('creation_source_ORG_INVITE', 0.014720276488196009),
 ('creation_source_PERSONAL_PROJECTS', 0.2638041972655051),
 ('creation_source_SIGNUP', 0.009058272088754258),
 ('creation_source_SIGNUP_GOOGLE_AUTH', 0.07818969117428781)]